# Analysis of the results

In [13]:
import sys
sys.path.append("..")

from analysis.analysis_utils import extract_all_info_from_hdf5
from analysis.dataset import Dataset
from plot.dataset_plot import plot_hist_by_phenotype
import matplotlib.pyplot as plt
import numpy as np

In [10]:
filepath = r"..\out\last\E3O2P21R12T17_S75_T0.7_M1000_N500000.h5"
adj_matrices, parameters = extract_all_info_from_hdf5(filepath)
niter = parameters["Number of interaction"]

In [12]:
local_dt = Dataset('local')
local_dt.init_with_matrices(adj_matrices, parameters, niter)

In [ ]:
plot_hist_by_phenotype(local_dt, 'pPD')
plt.show()